In [1]:
import bs4 as bs
import urllib.request
import pandas as pd
import numpy as np


url='https://www.11v11.com/competitions/premier-league/2021/matches/'
source = urllib.request.urlopen(url).read()


#page = urllib2.urlopen(url).read()
soup = bs.BeautifulSoup(source)

In [2]:
table = soup.find('table')

rows = list()
for row in table.findAll("tr"):
    rows.append(row)


In [3]:
date = []
home = []
away = []
score = []
for row in rows:
    home_start = str(row).find('"home">') + 7
    home_end = str(row).find('</td', home_start)
    home.append(str(row)[home_start:home_end])
    
    away_start = str(row).rfind('</a></td><td>') + 13
    away_end = str(row).rfind('</td><td></td></tr>')
    away.append(str(row)[away_start:away_end])
    
    score_start = str(row).rfind('"view match details">') + 21
    score_end = str(row).rfind('</a></td><td>')
    score.append(str(row)[score_start:score_end])
    
    date_start = 8
    date_end = str(rows[2]).find('</td><td')
    date.append(str(row)[date_start:date_end])

In [4]:
results = pd.DataFrame({
    'Date': date,
    'Season': 2020,
    'home': home,
    'visitor': away,
    'FT': score
}).assign(
    header = lambda x: ['colspan' in r for r in x.Date]
)

results = results[
    results.header == False
].drop(
    columns='header'
).assign(
    hgoal = lambda x: x.FT.str.split(':', expand=True)[0],
    vgoal = lambda x: x.FT.str.split(':', expand=True)[1],
    result = lambda x: np.where(x.hgoal > x.vgoal, 'H',
                               np.where(x.hgoal == x.vgoal, 'D', 'A'))
)

results['hgoal'] = np.where(results.hgoal == '', np.nan, results.hgoal).astype('Float64')
results['vgoal'] = np.where(results.vgoal == '', np.nan, results.vgoal).astype('Float64')
results['result'] = np.where(results.isnull()['hgoal'], np.nan, results.result)
results['goaldif'] = results.hgoal - results.vgoal
results['Date'] = pd.to_datetime(results.Date)

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.


In [5]:
results.tail()

,Date,Season,home,visitor,FT,hgoal,vgoal,result,goaldif
203,2021-01-30,2020,Crystal Palace,Wolverhampton Wanderers,:,NaN,NaN,NaN,NaN
204,2021-01-30,2020,Everton,Newcastle United,:,NaN,NaN,NaN,NaN
205,2021-01-30,2020,Manchester City,Sheffield United,:,NaN,NaN,NaN,NaN
206,2021-01-30,2020,Southampton,Aston Villa,:,NaN,NaN,NaN,NaN
207,2021-01-30,2020,West Bromwich Albion,Fulham,:,NaN,NaN,NaN,NaN


In [6]:
results.to_csv('/Users/darrelr/Documents/pl/data/england_2020.csv', index=False)